## Cohere Command-R and Rerank

In [38]:
### Set API Keys
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_tracing_v2 = os.getenv("LANGCHAIN_TRACING_V2")
cohere_api_key = os.getenv("COHERE_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [26]:
#Import Libraries
from langchain.vectorstores.deeplake import DeepLake
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_cohere import ChatCohere
from langchain_cohere import CohereEmbeddings
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM


In [40]:
#Initializing CohereEmbeddings and DeepLake
embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key= cohere_api_key,
)

docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embeddings,
    verbose=False,
    num_workers=4,
)


Deep Lake Dataset in deeplake_vstore already exists, loading from the storage


In [27]:
#Ingesting Document into Vector Store
file = "/home/martin/Documents/LegalAI-RAG-Contract-Advisor/data/Raptor_Contract.pdf"
loader = PyPDFLoader(file_path=file)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
pages = loader.load()
chunks = text_splitter.split_documents(pages)
_ = docstore.add_documents(documents=chunks)

Creating 314 embeddings in 1 batches of size 314:: 100%|██████████| 1/1 [00:20<00:00, 20.56s/it]


In [41]:
#Initializing DeepLake Vector Store and ChatCohere 
docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embeddings,
    verbose=False,
    read_only=True,
    num_workers=4,
)

llm = ChatCohere(
    model="command",
    cohere_api_key=cohere_api_key,
    temperature=0.7,
)


Deep Lake Dataset in deeplake_vstore already exists, loading from the storage


In [29]:
#Creating Retriever Object
retriever = docstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "fetch_k": 20,
        "k": 10,
    }
)

In [30]:
#Defining Prompt Template and Initializing RetrievalQA Chain
prompt_template = """You are an intelligent chatbot that can answer user's 
queries. You will be provided with Relevant context based on the user's queries. 
Your task is to analyze user's query and generate response for the query 
utiliing the context. Make sure to suggest one similar follow-up question based 
on the context for the user to ask.

NEVER generate response to queries for which there is no or irrelevant context.

Context: {context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
chain_type_kwargs = {"prompt": PROMPT, "verbose": False}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs=chain_type_kwargs,
)

In [33]:
#Generating a Response to a Query
query = "Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?"
response = qa.invoke({"query": query})
result = response["result"]
print(result)

Under the agreement, each Seller is individually responsible for any breach of representations and warranties they have made to the Buyer. This means that if a Seller breaches a representation or warranty, they are liable for the consequences of that breach. The extent of this liability would depend on the specific terms outlined in the agreement and the nature of the breach.

Follow-up question: How are breaches of representations and warranties typically resolved in such agreements?


#### Implementing Cohere’s Reranker Model

In [34]:
#Importing Modules
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [35]:
#Creating the Reranker Object and Compression Retriever
cohere_rerank = CohereRerank(
    cohere_api_key="",
    model="rerank-english-v3.0",
)
docstore = DeepLake(
    dataset_path="deeplake_vstore",
    embedding=embeddings,
    verbose=False,
    read_only=True,
    num_workers=4,
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank,
    base_retriever=docstore.as_retriever(
        search_type="similarity",
        search_kwargs={
            "fetch_k": 20,
            "k": 15,
        },
    ),
)

Deep Lake Dataset in deeplake_vstore already exists, loading from the storage


In [36]:
prompt_template = """You are an intelligent chatbot that can answer user's 
queries. You will be provided with Relevant context based on the user's queries. 
Your task is to analyze user's query and generate response for the query 
utiliing the context. Make sure to suggest one similar follow-up question based 
on the context for the user to ask.

NEVER generate response to queries for which there is no or irrelevant context.

Context: {context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT, "verbose": False}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs=chain_type_kwargs,
)

In [37]:
#Generating a Response to a Query
query = "Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties?"
response = qa.invoke({"query": query})
result = response["result"]
print(result)

Under the agreement, each Seller is individually responsible for their representations and warranties. They are required to indemnify the Sellers' Representative from any losses arising out of serving in that capacity, except for losses caused by gross negligence, bad faith, or willful misconduct. The extent of their responsibility depends on the specific circumstances of the breach and is determined by their Pro Rata Percentage. 

Follow-up question: How are Pro Rata Percentages determined in the context of the agreement between the Sellers and the Buyers?
